In [2]:
%pip install transformers
%pip install sklearn

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'C:\Users\giris\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'C:\Users\giris\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip' command.


In [3]:
from transformers import BertModel, BertTokenizer
import torch
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity


def text_summarizer(text, num_sentences=3):
    # Load pre-trained BERT model and tokenizer
    model_name = 'bert-base-uncased'
    tokenizer = BertTokenizer.from_pretrained(model_name)
    model = BertModel.from_pretrained(model_name)

    # Tokenize input text
    tokenized_text = tokenizer.tokenize(text)
    indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
    # All tokens belong to a single segment in this case
    segments_ids = [1] * len(tokenized_text)

    # Convert inputs to PyTorch tensors
    tokens_tensor = torch.tensor([indexed_tokens])
    segments_tensors = torch.tensor([segments_ids])

    # Get BERT embeddings for the input text
    with torch.no_grad():
        outputs = model(tokens_tensor, segments_tensors)
        encoded_layers = outputs[0]

    # Calculate sentence embeddings by averaging token embeddings for each sentence
    token_embeddings = np.array(encoded_layers[0])
    sentence_embeddings = []
    current_sentence = []
    for token in token_embeddings:
        if len(current_sentence) < 512:  # Max token limit per sentence for BERT
            current_sentence.append(token)
        else:
            sentence_embeddings.append(np.mean(current_sentence, axis=0))
            current_sentence = [token]

    if current_sentence:
        sentence_embeddings.append(np.mean(current_sentence, axis=0))

    # Calculate similarities between sentence embeddings
    similarity_matrix = cosine_similarity(
        sentence_embeddings, sentence_embeddings)

    # Create a similarity ranking for sentences
    sentence_similarity_graph = similarity_matrix.argsort(axis=1)
    sentence_similarity_graph = np.fliplr(sentence_similarity_graph)

    # Generate summary based on the top-ranked sentences
    summary = ''
    sentence_count = 0
    for row in sentence_similarity_graph:
        if sentence_count < num_sentences:
            summary += ' '.join(tokenized_text[row[0]]) + ' '
            sentence_count += 1
        else:
            break

    return summary.strip()

C:\Users\giris\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\LocalCache\local-packages\Python38\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
# Define a sample text for summarization
sample_text = """
Natural language processing (NLP) is a field of artificial intelligence 
(AI) that focuses on enabling computers to understand, interpret, and 
generate human language content. NLP techniques are used for various tasks 
such as language translation, sentiment analysis, text summarization, and 
more. One popular approach to NLP involves using pre-trained models like 
BERT, which is a powerful transformer-based model developed by Google. BERT 
has been widely used in natural language understanding and can be adapted 
for different NLP tasks through fine-tuning or transfer learning.
"""

# Call the text summarizer function
summary = text_summarizer(sample_text, num_sentences=2)

# Print the original text and the generated summary
print("Original Text:\n", sample_text)
print("\nGenerated Summary:\n", summary)

tokenizer_config.json: 100%|██████████| 28.0/28.0 [00:00<00:00, 5.58kB/s]
C:\Users\giris\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\LocalCache\local-packages\Python38\site-packages\huggingface_hub\file_download.py:147: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\giris\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warn

Original Text:
 
Natural language processing (NLP) is a field of artificial intelligence 
(AI) that focuses on enabling computers to understand, interpret, and 
generate human language content. NLP techniques are used for various tasks 
such as language translation, sentiment analysis, text summarization, and 
more. One popular approach to NLP involves using pre-trained models like 
BERT, which is a powerful transformer-based model developed by Google. BERT 
has been widely used in natural language understanding and can be adapted 
for different NLP tasks through fine-tuning or transfer learning.


Generated Summary:
 n a t u r a l
